#### PIPE - 4:    
This pipe covers the following : 
1. All features of pipe 3
2. Cropping images for ANRP use
3. Correcting blur in images for ANRP use
4. Enhancing images for ANRP use

In [1]:
import cv2
import numpy as np
import pandas as pd
import torch
import time
from ultralytics import YOLO
from pathlib import Path
from boxmot import StrongSORT
from utils.utils import *

down = {}
up = {}
text_color = (0, 0, 0)  # Black color for text
yellow_color = (0, 255, 255)  # Yellow color for background
red_color = (0, 0, 255)  # Red color for lines
blue_color = (255, 0, 0)  # Blue color for lines

counter_down = []
counter_up = []

first_boundry_y=170

red_line_y = 198

blue_line_y = 280

offset = 6
# Specify the start and end points of the line
start_point = (300, 198)
end_point = (300, 280)

all_id_tracker=[]
voilation_id_tracker=[]
violation_frame_tracker={}
tracker=tracker_init(cuda_device=torch.cuda.is_available(),cuda_device_number=1)
model=YOLO('yolov8s.pt')
video_path='/media/hlink/hd/test_videos/testx_vid_1.mp4'
vid = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_main = cv2.VideoWriter('output_strongs.avi', fourcc, 20.0, (1020, 500))
frame_count=0
all_frames_record_path=create_directory('all_frames_record')
violation_frames_record_path=create_directory('all_violation_record')

while True:    
    ret, im = vid.read()
    
    if not ret:
        break
    
    im=cv2.resize(im,(1020, 500))
    
    
    results=model.predict(im,conf=0.4,verbose=False,device=[1],classes=[2,3])
    im=line_plotter(frame=im,x_start=300,x_end=774,y=198,line_color=red_color,line_name='boundry 1',line_thickness=2,text_color=text_color)
    im=line_plotter(frame=im,x_start=300,x_end=800,y=280,line_color=blue_color,line_name='boundry 2',line_thickness=2,text_color=text_color)
    
    # Specify the color of the line in BGR format (here, it's white)
    color = (255, 255, 255)
    
    # Draw the vertical line on the image
    thickness = 2  # You can adjust the thickness as needed
    cv2.line(im, start_point, end_point, color, thickness)
    
    # im=cv2.rectangle(im, (300, 198), (800, 280), (255, 255, 255), 2)
    
    px,conf=prediction_coordinated_hadler(results)
    dets = []
    # Experimenting
    dets=tracker_element_creator(dets,px,conf)        
    
    # print('out_from')
    
    dets = np.array(dets)
    # print(dets)
    if len(dets) > 0:
        tracks = tracker.update(dets, im) # --> M X (x, y, x, y, id, conf, cls, ind)
        # print('tracks',tracks)    
        for track in tracks:
                im,object_id,cx,cy,bbox_list=plot_tracks(track,im)
               
                all_id_tracker.append(object_id)
                # Adding file path for csv
                csv_file_path=detection_coordinate_write(frame_count,object_id,bbox_list,'all_frame_detection_detail.csv')
            
                if red_line_y<(cy+offset) and red_line_y > (cy-offset):
                    # print("entered if 1")
                    down[object_id]=time.time()
            
                if object_id in down:
                    # print("entered if 2")
               
                    if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
                         # print("entered if 3")
                         elapsed_time=time.time() - down[object_id] 
                         # print("entered if 4")
                         if counter_down.count(object_id)==0:
                            counter_down.append(object_id) 
                            distance = 9
                            est_speed=speed_calculator_kmph(distance,elapsed_time)
                            if est_speed>20:
                                print(f'violation_detected : {object_id}')
                                voilation_id_tracker.append(object_id)
                                violation_frame_tracker[object_id]=frame_count
                                violation_frame_writer_op=frame_writer(frame_count,im,violation_frames_record_path)
                                violation_csv_file_path=detection_coordinate_write(frame_count,object_id,bbox_list,'violation_frame_detection_detail.csv')
                                
                            print(f"for {object_id} speed is -> {est_speed}")
    
    all_frame_writer_op=frame_writer(frame_count,im,all_frames_record_path)    
    frame_count+=1
    out_main.write(im)
    
    # # break on pressing q or spaceq
    cv2.imshow('BoxMOT detection', im)     
    # key = cv2.waitKey(25) & 0xFF
    
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()
print(all_id_tracker)
print(voilation_id_tracker)

2024-05-10 15:17:31.642 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "models/osnet_x0_25_msmt17.pt"


violation_detected : 5
for 5 speed is -> 32
violation_detected : 10
for 10 speed is -> 35
violation_detected : 28
for 28 speed is -> 32
violation_detected : 34
for 34 speed is -> 35
violation_detected : 72
for 72 speed is -> 34
violation_detected : 88
for 88 speed is -> 26
violation_detected : 107
for 107 speed is -> 37
violation_detected : 113
for 113 speed is -> 36
violation_detected : 174
for 174 speed is -> 26
for 192 speed is -> 17
violation_detected : 169
for 169 speed is -> 22
violation_detected : 180
for 180 speed is -> 32
violation_detected : 213
for 213 speed is -> 38
violation_detected : 214
for 214 speed is -> 45
violation_detected : 220
for 220 speed is -> 28
violation_detected : 223
for 223 speed is -> 30
for 232 speed is -> 10
violation_detected : 239
for 239 speed is -> 46
violation_detected : 241
for 241 speed is -> 43
violation_detected : 274
for 274 speed is -> 40
violation_detected : 263
for 263 speed is -> 31
violation_detected : 315
for 315 speed is -> 43
violatio

In [2]:
def frame_order_checker(arr):
    """
    Used to check the order of frames appended in the violation json
    """
    order_maintained=True
    for cnt,i in enumerate(arr):
        if str(cnt+1) >= str(len(arr)):
            # print("finished")
            break
        if int(arr[cnt][0]) <= int(arr[cnt+1][0]):
            continue
        else:
            order_maintained=False
    return order_maintained


In [3]:
def json_frame_order_checker(main_json):
    """
    Used to check the order of frames
    """
    maintainer=True
    for keys_ in main_json:
        order_maintained=frame_order_checker(main_json[keys_])
        print(f"order_maintainer - {order_maintained}")
        if not order_maintained:
            maintainer=False
    return maintainer

In [4]:
%%time
main_violation_json=voilation_capture_json_creator(voilation_id_tracker,csv_file_path)

CPU times: user 49.3 ms, sys: 4.4 ms, total: 53.7 ms
Wall time: 52.7 ms


In [5]:
%%time
maintanied=json_frame_order_checker(main_violation_json)

order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
CPU times: user 150 µs, sys: 255 µs, total: 405 µs
Wall time: 404 µs


In [6]:
%%time
if maintanied:
    evidance_img_dir_paths,evidance_clip_dir_paths=evidance_directories_creator(main_violation_json)

CPU times: user 1.11 ms, sys: 3.39 ms, total: 4.5 ms
Wall time: 4.1 ms


In [7]:
%%time
if maintanied:
    evidance_img_dir_paths=evidance_img_separator(evidance_img_dir_paths,all_frames_record_path,main_violation_json)

CPU times: user 180 ms, sys: 1.98 s, total: 2.16 s
Wall time: 2.17 s


In [8]:
%%time
if maintanied:
    video_clip_creator(voilation_id_tracker,violation_frame_tracker,evidance_clip_dir_paths,all_frames_record_path)

CPU times: user 11.3 s, sys: 2.06 s, total: 13.3 s
Wall time: 13.4 s


In [9]:
print(maintanied)

True


In [10]:
%%time
cropped_images_pathss=evidance_cropper(evidance_img_dir_paths,main_violation_json)

CPU times: user 27.1 s, sys: 779 ms, total: 27.9 s
Wall time: 28.1 s


In [43]:
%%time
deblurred_image_paths=deblur_images(cropped_images_pathss,main_violation_json)

CPU times: user 609 ms, sys: 415 ms, total: 1.02 s
Wall time: 1.08 s


In [46]:
%%time
enhance_images(deblurred_image_paths,main_violation_json)

/home/hlink/workspace/learning/boxmot/10_05_2024_infenrence_output/enhanced_images/5/23.jpg
/home/hlink/workspace/learning/boxmot/10_05_2024_infenrence_output/enhanced_images/5/24.jpg
/home/hlink/workspace/learning/boxmot/10_05_2024_infenrence_output/enhanced_images/5/25.jpg
/home/hlink/workspace/learning/boxmot/10_05_2024_infenrence_output/enhanced_images/5/26.jpg
/home/hlink/workspace/learning/boxmot/10_05_2024_infenrence_output/enhanced_images/5/27.jpg
/home/hlink/workspace/learning/boxmot/10_05_2024_infenrence_output/enhanced_images/5/28.jpg
/home/hlink/workspace/learning/boxmot/10_05_2024_infenrence_output/enhanced_images/5/29.jpg
/home/hlink/workspace/learning/boxmot/10_05_2024_infenrence_output/enhanced_images/5/30.jpg
/home/hlink/workspace/learning/boxmot/10_05_2024_infenrence_output/enhanced_images/5/31.jpg
/home/hlink/workspace/learning/boxmot/10_05_2024_infenrence_output/enhanced_images/5/32.jpg
/home/hlink/workspace/learning/boxmot/10_05_2024_infenrence_output/enhanced_imag

['/home/hlink/workspace/learning/boxmot/10_05_2024_infenrence_output/enhanced_images/852']

In [49]:
help(enhance_images)

Help on function enhance_images in module __main__:

enhance_images(cropped_images_pathss, main_violation_json)

